In [4]:
!pip -q install chess ipywidgets pandas numpy scikit-learn tensorflow

In [5]:
# -------------------------------
#    ML MODEL TRAINING SECTION
# -------------------------------

import pandas as pd
import numpy as np
import chess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import os

print("Loading chess evaluation dataset...")
# Load the dataset (using a sample for faster training - adjust as needed)
df = pd.read_csv('archive/chessData.csv', nrows=500000)  # Using 500k rows for reasonable training time
print(f"Dataset loaded: {len(df)} positions")
print(f"Evaluation range: {df['Evaluation'].min()} to {df['Evaluation'].max()}")

# Display sample data
print("\nSample data:")
print(df.head(3))

Loading chess evaluation dataset...
Dataset loaded: 500000 positions
Evaluation range: #+0 to 0

Sample data:
                                                 FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9


In [6]:
# -------------------------------
#    FEN TO FEATURES CONVERSION
# -------------------------------

def fen_to_features(fen_string):
    """
    Convert a FEN position to a numerical feature vector.
    
    Features include:
    - 64 squares × 12 piece types (6 pieces × 2 colors) = 768 features (one-hot encoded board)
    - 1 feature for side to move (0=white, 1=black)
    - 4 features for castling rights (KQkq)
    - 1 feature for en passant availability
    - Total: 774 features
    """
    try:
        board = chess.Board(fen_string)
    except:
        # Return zero vector for invalid FEN
        return np.zeros(774)
    
    features = []
    
    # Board representation: 64 squares × 12 piece types
    piece_map = {
        (chess.PAWN, chess.WHITE): 0,
        (chess.KNIGHT, chess.WHITE): 1,
        (chess.BISHOP, chess.WHITE): 2,
        (chess.ROOK, chess.WHITE): 3,
        (chess.QUEEN, chess.WHITE): 4,
        (chess.KING, chess.WHITE): 5,
        (chess.PAWN, chess.BLACK): 6,
        (chess.KNIGHT, chess.BLACK): 7,
        (chess.BISHOP, chess.BLACK): 8,
        (chess.ROOK, chess.BLACK): 9,
        (chess.QUEEN, chess.BLACK): 10,
        (chess.KING, chess.BLACK): 11,
    }
    
    # Create 64 × 12 board representation
    board_features = np.zeros(64 * 12)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_idx = piece_map[(piece.piece_type, piece.color)]
            board_features[square * 12 + piece_idx] = 1
    
    features.extend(board_features)
    
    # Side to move
    features.append(1 if board.turn == chess.BLACK else 0)
    
    # Castling rights
    features.append(1 if board.has_kingside_castling_rights(chess.WHITE) else 0)
    features.append(1 if board.has_queenside_castling_rights(chess.WHITE) else 0)
    features.append(1 if board.has_kingside_castling_rights(chess.BLACK) else 0)
    features.append(1 if board.has_queenside_castling_rights(chess.BLACK) else 0)
    
    # En passant
    features.append(1 if board.ep_square is not None else 0)
    
    return np.array(features, dtype=np.float32)

# Test the conversion function
test_fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1"
test_features = fen_to_features(test_fen)
print(f"Feature vector shape: {test_features.shape}")
print(f"Sample features (first 20): {test_features[:20]}")
print(f"Non-zero features: {np.count_nonzero(test_features)}")

Feature vector shape: (774,)
Sample features (first 20): [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Non-zero features: 37


In [17]:
# -------------------------------
#    DATA PREPROCESSING
# -------------------------------

print("Converting FEN positions to features...")
print("This may take a few minutes...")

# Convert all FEN positions to feature vectors
X = np.array([fen_to_features(fen) for fen in df['FEN']])

# Clean and convert evaluation scores to numeric
print("\nCleaning evaluation scores...")

def clean_evaluation(eval_str):
    """
    Convert evaluation string to numeric value.
    Handles formats like: '+56', '-9', '#3' (mate in 3), etc.
    """
    if pd.isna(eval_str):
        return 0
    
    eval_str = str(eval_str).strip()
    
    # Handle mate scores (e.g., '#3' means mate in 3)
    if '#' in eval_str:
        # Mate scores: assign large values
        mate_moves = eval_str.replace('#', '').replace('+', '').replace('-', '')
        try:
            moves = int(mate_moves)
            # Positive mate = white winning, negative = black winning
            if '-' in eval_str:
                return -10000 + moves * 10  # Closer mate = more negative
            else:
                return 10000 - moves * 10   # Closer mate = more positive
        except:
            return 0
    
    # Handle regular numeric evaluations
    try:
        # Remove '+' sign if present and convert to float
        return float(eval_str.replace('+', ''))
    except:
        return 0

# Apply cleaning function
y = df['Evaluation'].apply(clean_evaluation).values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Evaluation data type: {y.dtype}")
print(f"Sample evaluations: {y[:10]}")

# Clip extreme values to prevent outliers from dominating
y_clipped = np.clip(y, -2000, 2000)
print(f"\nEvaluation range before clipping: {y.min():.2f} to {y.max():.2f}")
print(f"Evaluation range after clipping: {y_clipped.min():.2f} to {y_clipped.max():.2f}")

# NORMALIZE evaluations to [0, 1] range for better training
# This maps [-2000, +2000] → [0, 1]
y_normalized = (y_clipped + 2000) / 4000
print(f"\nNormalized evaluation range: {y_normalized.min():.4f} to {y_normalized.max():.4f}")
print(f"Sample normalized values: {y_normalized[:5]}")

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_normalized, test_size=0.2, random_state=42
)

print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData preprocessing complete!")
print("Note: Evaluations are normalized to [0, 1] range for training.")

Converting FEN positions to features...
This may take a few minutes...

Cleaning evaluation scores...

Feature matrix shape: (500000, 774)
Target vector shape: (500000,)
Evaluation data type: float64
Sample evaluations: [-10.  56.  -9.  52. -26.  50.  10.  75.  52.  52.]

Evaluation range before clipping: -15291.00 to 10000.00
Evaluation range after clipping: -2000.00 to 2000.00

Normalized evaluation range: 0.0000 to 1.0000
Sample normalized values: [0.4975  0.514   0.49775 0.513   0.4935 ]

Training set size: 400000
Test set size: 100000

Data preprocessing complete!
Note: Evaluations are normalized to [0, 1] range for training.


In [18]:
# -------------------------------
#    NEURAL NETWORK MODEL
# -------------------------------

print("Building neural network model...")

# Create the model
model = keras.Sequential([
    layers.Input(shape=(774,)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output: single evaluation score
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Display model architecture
model.summary()

print("\nModel built successfully!")

Building neural network model...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 512)            │       396,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 569,345 (2.17 MB)

 Trainable params: 569,345 (2.17 MB)

 Non-trainable params: 0 (0.00 B)


Model built successfully!


In [19]:
# -------------------------------
#    MODEL TRAINING
# -------------------------------

print("Training the model...")
print("This will take several minutes depending on your hardware...\n")

# Early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=256,
    callbacks=[early_stopping],
    verbose=1
)

print("\nTraining complete!")

Training the model...
This will take several minutes depending on your hardware...

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0166 - mae: 0.0734 - val_loss: 0.0105 - val_mae: 0.0550
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0101 - mae: 0.0569 - val_loss: 0.0092 - val_mae: 0.0515
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0088 - mae: 0.0531 - val_loss: 0.0078 - val_mae: 0.0484
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0075 - mae: 0.0497 - val_loss: 0.0070 - val_mae: 0.0460
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0064 - mae: 0.0468 - val_loss: 0.0065 - val_mae: 0.0450
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0055 - mae: 0.0442 - val_loss: 0.0060 - val_mae: 0.0432
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0048 - mae: 0.0421 - val_loss: 0.0054 - val_mae: 0.0417
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0043 - mae: 

In [21]:
# -------------------------------
#    MODEL EVALUATION
# -------------------------------

print("Evaluating model on test set...")

# Evaluate on test set (normalized values)
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"\nTest Set Performance (Normalized Scale [0,1]):")
print(f"Mean Squared Error (MSE): {test_loss:.6f}")
print(f"Mean Absolute Error (MAE): {test_mae:.6f}")

# Convert back to centipawns for interpretability
test_mae_centipawns = test_mae * 4000  # Denormalize
test_rmse_centipawns = np.sqrt(test_loss) * 4000

print(f"\nTest Set Performance (Centipawn Scale):")
print(f"Mean Absolute Error (MAE): {test_mae_centipawns:.2f} centipawns ({test_mae_centipawns/100:.2f} pawns)")
print(f"Root Mean Squared Error (RMSE): {test_rmse_centipawns:.2f} centipawns ({test_rmse_centipawns/100:.2f} pawns)")

# Make predictions on a few test samples
sample_predictions_norm = model.predict(X_test_scaled[:5], verbose=0)
# Denormalize predictions and actuals
sample_predictions_cp = (sample_predictions_norm * 4000) - 2000
sample_actuals_cp = (y_test[:5] * 4000) - 2000

print(f"\nSample Predictions vs Actual (in centipawns):")
for i in range(5):
    pred = sample_predictions_cp[i][0]
    actual = sample_actuals_cp[i]
    error = abs(pred - actual)
    print(f"  Predicted: {pred:+7.1f} cp | Actual: {actual:+7.1f} cp | Error: {error:6.1f} cp")

# Save the model and scaler
print("\nSaving model and preprocessing objects...")
model.save('chess_eval_model.h5')
with open('chess_eval_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save normalization parameters for later use
normalization_params = {
    'min_value': -2000,
    'max_value': 2000,
    'range': 4000
}
with open('chess_eval_norm_params.pkl', 'wb') as f:
    pickle.dump(normalization_params, f)

print("Model saved as 'chess_eval_model.h5'")
print("Scaler saved as 'chess_eval_scaler.pkl'")
print("Normalization params saved as 'chess_eval_norm_params.pkl'")

Evaluating model on test set...



Test Set Performance (Normalized Scale [0,1]):
Mean Squared Error (MSE): 0.004056
Mean Absolute Error (MAE): 0.036282

Test Set Performance (Centipawn Scale):
Mean Absolute Error (MAE): 145.13 centipawns (1.45 pawns)
Root Mean Squared Error (RMSE): 254.74 centipawns (2.55 pawns)

Sample Predictions vs Actual (in centipawns):
  Predicted:   +19.6 cp | Actual:   +51.0 cp | Error:   31.4 cp
  Predicted:  +832.4 cp | Actual: +1076.0 cp | Error:  243.6 cp
  Predicted:   +65.7 cp | Actual:   -60.0 cp | Error:  125.7 cp
  Predicted:   +22.2 cp | Actual:   -63.0 cp | Error:   85.2 cp
  Predicted:  +151.9 cp | Actual:  +230.0 cp | Error:   78.1 cp

Saving model and preprocessing objects...
Model saved as 'chess_eval_model.h5'
Scaler saved as 'chess_eval_scaler.pkl'
Normalization params saved as 'chess_eval_norm_params.pkl'


In [25]:
# -------------------------------
#    LOAD ML MODEL FOR CHESS ENGINE
# -------------------------------

print("Loading trained ML model for chess evaluation...")

# Load the trained model and scaler
try:
    # Load with compile=False to avoid metric deserialization issues
    ml_model = keras.models.load_model('chess_eval_model.h5', compile=False)
    
    # Recompile the model with current Keras version
    ml_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    with open('chess_eval_scaler.pkl', 'rb') as f:
        ml_scaler = pickle.load(f)
    with open('chess_eval_norm_params.pkl', 'rb') as f:
        ml_norm_params = pickle.load(f)
    print("✓ ML model, scaler, and normalization params loaded successfully!")
    print(f"  Normalization range: [{ml_norm_params['min_value']}, {ml_norm_params['max_value']}] centipawns")
    ML_MODEL_AVAILABLE = True
except Exception as e:
    print(f"⚠ ML model not found. Error: {e}")
    print("  Please train the model first by running the cells above.")
    print("  Falling back to traditional evaluation.")
    ML_MODEL_AVAILABLE = False
    ml_model = None
    ml_scaler = None
    ml_norm_params = None

Loading trained ML model for chess evaluation...
✓ ML model, scaler, and normalization params loaded successfully!
  Normalization range: [-2000, 2000] centipawns


In [26]:
import math
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict

import chess
import ipywidgets as W
from IPython.display import display, HTML

# -------------------------------
#           ENGINE
# -------------------------------

PIECE_VALUES = {
    chess.PAWN:   100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK:   500,
    chess.QUEEN:  900,
    chess.KING:   0,  # handled via mate scores
}

PAWN_TABLE = [
      0,  0,  0,  0,  0,  0,  0,  0,
     50, 50, 50, 50, 50, 50, 50, 50,
     10, 10, 20, 30, 30, 20, 10, 10,
      5,  5, 10, 25, 25, 10,  5,  5,
      0,  0,  0, 20, 20,  0,  0,  0,
      5, -5,-10,  0,  0,-10, -5,  5,
      5, 10, 10,-20,-20, 10, 10,  5,
      0,  0,  0,  0,  0,  0,  0,  0,
]
KNIGHT_TABLE = [
    -50,-40,-30,-30,-30,-30,-40,-50,
    -40,-20,  0,  5,  5,  0,-20,-40,
    -30,  5, 10, 15, 15, 10,  5,-30,
    -30,  0, 15, 20, 20, 15,  0,-30,
    -30,  5, 15, 20, 20, 15,  5,-30,
    -30,  0, 10, 15, 15, 10,  0,-30,
    -40,-20,  0,  0,  0,  0,-20,-40,
    -50,-40,-30,-30,-30,-30,-40,-50,
]
BISHOP_TABLE = [
    -20,-10,-10,-10,-10,-10,-10,-20,
    -10,  5,  0,  0,  0,  0,  5,-10,
    -10, 10, 10, 10, 10, 10, 10,-10,
    -10,  0, 10, 10, 10, 10,  0,-10,
    -10,  5,  5, 10, 10,  5,  5,-10,
    -10,  0,  5, 10, 10,  5,  0,-10,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -20,-10,-10,-10,-10,-10,-10,-20,
]
ROOK_TABLE = [
     0,  0,  0,  5,  5,  0,  0,  0,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
     5, 10, 10, 10, 10, 10, 10,  5,
     0,  0,  0,  0,  0,  0,  0,  0,
]
QUEEN_TABLE = [
    -20,-10,-10, -5, -5,-10,-10,-20,
    -10,  0,  5,  0,  0,  0,  0,-10,
    -10,  5,  5,  5,  5,  5,  0,-10,
     -5,  0,  5,  5,  5,  5,  0, -5,
      0,  0,  5,  5,  5,  5,  0, -5,
    -10,  0,  5,  5,  5,  5,  0,-10,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -20,-10,-10, -5, -5,-10,-10,-20,
]
KING_TABLE = [
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -20,-30,-30,-40,-40,-30,-30,-20,
    -10,-20,-20,-20,-20,-20,-20,-10,
     20, 20,  0,  0,  0,  0, 20, 20,
     20, 30, 10,  0,  0, 10, 30, 20,
]
PST = {
    chess.PAWN:   PAWN_TABLE,
    chess.KNIGHT: KNIGHT_TABLE,
    chess.BISHOP: BISHOP_TABLE,
    chess.ROOK:   ROOK_TABLE,
    chess.QUEEN:  QUEEN_TABLE,
    chess.KING:   KING_TABLE
}
MATE_VALUE = 10_000

# -------------------------------
#    ML-BASED EVALUATION FUNCTION
# -------------------------------

def evaluate_board_ml(board: chess.Board) -> int:
    """
    ML-based board evaluation using the trained neural network.
    Returns evaluation in centipawns from White's perspective.
    """
    # Handle game over positions
    if board.is_game_over():
        outcome = board.outcome()
        if outcome is None or outcome.winner is None:
            return 0
        return MATE_VALUE if outcome.winner == chess.WHITE else -MATE_VALUE
    
    # Convert board to features
    fen = board.fen()
    features = fen_to_features(fen)
    features_scaled = ml_scaler.transform(features.reshape(1, -1))
    
    # Get prediction from model (normalized [0,1])
    prediction_normalized = ml_model.predict(features_scaled, verbose=0)[0][0]
    
    # Denormalize: [0, 1] → [-2000, +2000] centipawns
    prediction_centipawns = (prediction_normalized * ml_norm_params['range']) + ml_norm_params['min_value']
    
    # Return as integer centipawns
    return int(prediction_centipawns)

# -------------------------------
#    TRADITIONAL EVALUATION (FALLBACK)
# -------------------------------

def evaluate_board_raw(board: chess.Board) -> int:
    """
    Traditional evaluation using piece-square tables.
    This is used as fallback if ML model is not available.
    """
    if board.is_game_over():
        outcome = board.outcome()
        if outcome is None or outcome.winner is None:
            return 0
        return MATE_VALUE if outcome.winner == chess.WHITE else -MATE_VALUE

    score = 0
    for sq, piece in board.piece_map().items():
        base = PIECE_VALUES[piece.piece_type]
        if piece.color == chess.WHITE:
            score += base + PST[piece.piece_type][sq]
        else:
            score -= base + PST[piece.piece_type][chess.square_mirror(sq)]
    return score

# -------------------------------
#    UNIFIED EVALUATION FUNCTION
# -------------------------------

def evaluate_board(board: chess.Board) -> int:
    """
    Unified evaluation function that uses ML if available, 
    otherwise falls back to traditional evaluation.
    """
    if ML_MODEL_AVAILABLE:
        return evaluate_board_ml(board)
    else:
        return evaluate_board_raw(board)

def eval_for_color(board: chess.Board, ai_color: chess.Color) -> int:
    raw = evaluate_board(board)
    return raw if ai_color == chess.WHITE else -raw

def move_order_key(board: chess.Board, move: chess.Move) -> int:
    score = 0
    if board.is_capture(move):
        captured = board.piece_type_at(move.to_square)
        attacker = board.piece_type_at(move.from_square)
        if captured:
            score += 10 * PIECE_VALUES[captured] - PIECE_VALUES.get(attacker, 0)
        else:
            score += 500
    if move.promotion:
        score += PIECE_VALUES.get(move.promotion, 0)
    board.push(move)
    if board.is_check():
        score += 50
    board.pop()
    return score

def ordered_moves(board: chess.Board) -> List[chess.Move]:
    ms = list(board.legal_moves)
    ms.sort(key=lambda m: move_order_key(board, m), reverse=True)
    return ms

def minimax(board: chess.Board, depth: int, alpha: int, beta: int, ai_color: chess.Color) -> Tuple[int, Optional[chess.Move]]:
    if depth == 0 or board.is_game_over():
        return eval_for_color(board, ai_color), None

    maximizing = (board.turn == ai_color)
    best_move = None

    if maximizing:
        best = -math.inf
        for mv in ordered_moves(board):
            board.push(mv)
            val, _ = minimax(board, depth - 1, alpha, beta, ai_color)
            board.pop()
            if val > best:
                best, best_move = val, mv
            alpha = max(alpha, best)
            if beta <= alpha:
                break
        return int(best), best_move
    else:
        best = math.inf
        for mv in ordered_moves(board):
            board.push(mv)
            val, _ = minimax(board, depth - 1, alpha, beta, ai_color)
            board.pop()
            if val < best:
                best, best_move = val, mv
            beta = min(beta, best)
            if beta <= alpha:
                break
        return int(best), best_move

def best_move(board: chess.Board, depth: int, ai_color: chess.Color) -> Optional[chess.Move]:
    score, mv = minimax(board, depth, -math.inf, math.inf, ai_color)
    return mv

# -------------------------------
#           UI (ipywidgets)
# -------------------------------

# Piece glyphs
GLYPH = {
    'P':'♙','N':'♘','B':'♗','R':'♖','Q':'♕','K':'♔',
    'p':'♟','n':'♞','b':'♝','r':'♜','q':'♛','k':'♚',
}
LIGHT = '#F0D9B5'
DARK  = '#B58863'
SEL   = '#f6f67a'
TARGET= '#b9e6a1'
CAPT  = '#f5a3a3'

# Optional: bump button font-size
display(HTML("<style>.widget-button{font-size:22px !important;}</style>"))

@dataclass
class GameState:
    board: chess.Board
    ai_color: Optional[chess.Color]  # None for 2-player mode
    depth: int
    orientation_white: bool  # True = white-bottom view

    def status_text(self) -> str:
        eval_mode = "ML" if ML_MODEL_AVAILABLE else "Traditional"
        if self.board.is_game_over():
            outcome = self.board.outcome()
            if outcome is None or outcome.winner is None:
                return f"Game over · Draw [{eval_mode}]"
            return f"Game over · " + ("White wins" if outcome.winner == chess.WHITE else "Black wins") + f" [{eval_mode}]"
        who = "White" if self.board.turn == chess.WHITE else "Black"
        check = " · Check!" if self.board.is_check() else ""
        eval_score = evaluate_board(self.board)
        return f"Turn: {who}{check} · Eval: {eval_score:+d} [{eval_mode}]"

class ChessApp:
    def __init__(self):
        # Controls
        self.mode = W.ToggleButtons(options=[('Vs AI','ai'),('Two Players','2p')], value='ai', description='Mode:')
        self.color_sel = W.ToggleButtons(options=[('White','white'),('Black','black')], value='white', description='You:')
        self.depth = W.IntSlider(value=3, min=2, max=5, step=1, description='AI depth:')
        self.start_btn = W.Button(description='Start / Reset', button_style='primary')
        self.undo_btn = W.Button(description='Undo')
        self.flip_btn = W.Button(description='Flip')
        eval_mode = "ML-Enhanced" if ML_MODEL_AVAILABLE else "Traditional"
        self.status = W.HTML(f"<b>Click Start / Reset to begin. [{eval_mode} Evaluation]</b>")
        self.log = W.HTML("")
        self.log_box = W.VBox([W.HTML("<b>Moves</b>"), W.Box([self.log], layout=W.Layout(max_height='220px', overflow='auto'))])

        # Promotion chooser (hidden until needed)
        self.promo_box = W.HBox([])
        self._promo_pending = None  # (from_sq, to_sq)

        # Board grid
        self.grid_box = W.GridBox(children=[], layout=W.Layout(grid_template_columns='repeat(8, 46px)', grid_gap='0px'))
        self.sq_buttons: Dict[int, W.Button] = {}  # square -> button
        self.selected_sq: Optional[int] = None
        self.legal_from_selected: List[chess.Move] = []

        # State
        self.state: Optional[GameState] = None

        # Wire controls
        self.start_btn.on_click(self.on_start)
        self.undo_btn.on_click(self.on_undo)
        self.flip_btn.on_click(self.on_flip)
        self.mode.observe(self.on_mode_change, 'value')

        # Build panel
        top = W.HBox([self.mode, self.color_sel, self.depth, self.start_btn, self.undo_btn, self.flip_btn])
        self.ui = W.VBox([top, self.status, self.promo_box, W.HBox([self.grid_box, self.log_box])])

        self._build_empty_grid()
        display(self.ui)

    # ---------- helpers ----------
    def _square_to_rc(self, sq: int) -> Tuple[int,int]:
        """Return (row, col) on the 8x8 grid for a given 0..63 square, given orientation."""
        file = chess.square_file(sq)
        rank = chess.square_rank(sq)
        if self.state and self.state.orientation_white:
            row = 7 - rank
            col = file
        else:
            row = rank
            col = 7 - file
        return row, col

    def _rc_to_square(self, row: int, col: int) -> int:
        """Return 0..63 square for grid (row, col)."""
        if self.state and self.state.orientation_white:
            rank = 7 - row
            file = col
        else:
            rank = row
            file = 7 - col
        return chess.square(file, rank)

    def _build_empty_grid(self):
        self.grid_box.children = ()
        self.sq_buttons.clear()
        children = []
        for row in range(8):
            for col in range(8):
                sq = self._rc_to_square(row, col)
                light = (row + col) % 2 == 0
                b = W.Button(description=' ', layout=W.Layout(width='46px', height='46px', padding='0'),
                             tooltip=chess.square_name(sq))
                b.style.button_color = LIGHT if light else DARK
                b.on_click(self._make_square_click(sq))
                self.sq_buttons[sq] = b
                children.append(b)
        self.grid_box.children = tuple(children)

    def _render_board(self):
        board = self.state.board
        # Clear any highlights if selection disappeared
        if self.selected_sq is not None and (self.selected_sq not in [m.from_square for m in board.legal_moves]):
            self.selected_sq = None
            self.legal_from_selected = []
        # Set base colors & glyphs
        for sq, btn in self.sq_buttons.items():
            row, col = self._square_to_rc(sq)
            light = (row + col) % 2 == 0
            btn.style.button_color = LIGHT if light else DARK
            piece = board.piece_at(sq)
            btn.description = GLYPH.get(piece.symbol(), ' ') if piece else ' '
        # Highlight selection and legal targets
        if self.selected_sq is not None:
            self.sq_buttons[self.selected_sq].style.button_color = SEL
            targets = [m.to_square for m in self.legal_from_selected]
            for m in self.legal_from_selected:
                tgt = m.to_square
                btn = self.sq_buttons[tgt]
                btn.style.button_color = CAPT if self.state.board.is_capture(m) else TARGET

        # Update status
        self.status.value = f"<b>{self.state.status_text()}</b>"

    def _append_log(self, text: str):
        self.log.value += f"{text}<br>"
        # cap log length a bit
        if self.log.value.count("<br>") > 200:
            self.log.value = "<i>(trimmed)</i><br>" + "<br>".join(self.log.value.split("<br>")[-200:])

    def _make_square_click(self, sq: int):
        def handler(_):
            if self.state is None or self.state.board.is_game_over():
                return
            # If promotion dialog visible, ignore board clicks
            if self._promo_pending is not None:
                return

            board = self.state.board

            # Whose turn is allowed to move?
            if self.state.ai_color is not None:
                human_turn = (board.turn != self.state.ai_color)
                if not human_turn:
                    return  # wait for AI
            # In 2-player mode both can move

            piece = board.piece_at(sq)
            if self.selected_sq is None:
                # First click: must be a piece of side to move
                if piece is None or piece.color != board.turn:
                    return
                self.selected_sq = sq
                self.legal_from_selected = [m for m in board.legal_moves if m.from_square == sq]
                self._render_board()
                return
            else:
                # Second click: attempt move
                if sq == self.selected_sq:
                    self.selected_sq = None
                    self.legal_from_selected = []
                    self._render_board()
                    return

                # Allow reselecting another own piece
                if piece is not None and piece.color == board.turn:
                    self.selected_sq = sq
                    self.legal_from_selected = [m for m in board.legal_moves if m.from_square == sq]
                    self._render_board()
                    return

                # Try to move selected -> sq (with possible promotion)
                legal_targets = [m for m in self.legal_from_selected if m.to_square == sq]
                if not legal_targets:
                    return

                # Promotion handling: if any candidate has a promotion, ask
                promos = [m for m in legal_targets if m.promotion]
                if promos:
                    self._prompt_promotion(self.selected_sq, sq, promos)
                    return

                # Otherwise, there should be exactly one legal move
                mv = legal_targets[0]
                self._play_human_move(mv)
        return handler

    def _prompt_promotion(self, from_sq: int, to_sq: int, promo_moves: List[chess.Move]):
        # Show small chooser
        self._promo_pending = (from_sq, to_sq)
        opts = [('Queen','q'),('Rook','r'),('Bishop','b'),('Knight','n')]
        picker = W.ToggleButtons(options=opts, value='q', description='Promote:')
        ok_btn = W.Button(description='OK', button_style='success')
        cancel_btn = W.Button(description='Cancel')
        msg = W.HTML("")

        def on_ok(_):
            letter = picker.value
            promo_map = {'q':chess.QUEEN,'r':chess.ROOK,'b':chess.BISHOP,'n':chess.KNIGHT}
            promo_piece = promo_map[letter]
            # find matching legal move
            for m in promo_moves:
                if m.from_square == from_sq and m.to_square == to_sq and m.promotion == promo_piece:
                    self._clear_promo_ui()
                    self._play_human_move(m)
                    return
            msg.value = "<span style='color:#b91c1c'>Not a legal promotion.</span>"

        def on_cancel(_):
            self._clear_promo_ui()
            # keep selection so user can try again

        ok_btn.on_click(on_ok)
        cancel_btn.on_click(on_cancel)
        self.promo_box.children = [picker, ok_btn, cancel_btn, msg]

    def _clear_promo_ui(self):
        self.promo_box.children = []
        self._promo_pending = None

    # ---------- actions ----------
    def on_start(self, _):
        human_color = self.color_sel.value
        ai = (None if self.mode.value == '2p'
              else (chess.BLACK if human_color == 'white' else chess.WHITE))
        self.state = GameState(board=chess.Board(), ai_color=ai, depth=int(self.depth.value),
                               orientation_white=True if human_color=='white' else False)
        self.selected_sq = None
        self.legal_from_selected = []
        self._build_empty_grid()
        self.log.value = ""
        self._clear_promo_ui()

        # If AI plays first (AI = white)
        if self.state.ai_color == chess.WHITE and self.state.board.turn == chess.WHITE:
            mv = best_move(self.state.board, self.state.depth, self.state.ai_color)
            if mv:
                san = self.state.board.san(mv)
                self.state.board.push(mv)
                self._append_log(f"AI: {san}")
        self._render_board()

    def on_undo(self, _):
        if not self.state: return
        b = self.state.board
        undone = 0
        if len(b.move_stack): b.pop(); undone += 1
        if self.state.ai_color is not None and len(b.move_stack):
            # try undo a full pair vs AI
            b.pop(); undone += 1
        self.selected_sq = None
        self.legal_from_selected = []
        self._append_log(f"<i>Undid {undone} half-move(s).</i>")
        self._render_board()

    def on_flip(self, _):
        if not self.state: return
        self.state.orientation_white = not self.state.orientation_white
        self._build_empty_grid()
        self._render_board()

    def on_mode_change(self, change):
        # Enable/disable controls based on mode
        is_ai = change['new'] == 'ai'
        self.color_sel.disabled = not is_ai
        self.depth.disabled = not is_ai

    def _play_human_move(self, mv: chess.Move):
        b = self.state.board
        san = b.san(mv)
        b.push(mv)
        self._append_log(f"You: {san}")
        self.selected_sq = None
        self.legal_from_selected = []
        self._render_board()
        # If game not over and vs AI, let AI respond
        if not b.is_game_over() and self.state.ai_color is not None and b.turn == self.state.ai_color:
            reply = best_move(b, self.state.depth, self.state.ai_color)
            if reply:
                san2 = b.san(reply)
                b.push(reply)
                self._append_log(f"AI: {san2}")
                self._render_board()

# Display evaluation mode
print(f"\n{'='*60}")
if ML_MODEL_AVAILABLE:
    print("🤖 CHESS ENGINE WITH ML EVALUATION ACTIVE")
    print("   The AI will use the trained neural network to evaluate positions.")
    print(f"   Evaluations normalized to [0,1] during training")
    print(f"   Predictions denormalized to [{ml_norm_params['min_value']}, {ml_norm_params['max_value']}] centipawns")
else:
    print("⚠️  CHESS ENGINE WITH TRADITIONAL EVALUATION")
    print("   ML model not loaded. Using piece-square table evaluation.")
print(f"{'='*60}\n")

# Launch the app
ChessApp()


🤖 CHESS ENGINE WITH ML EVALUATION ACTIVE
   The AI will use the trained neural network to evaluate positions.
   Evaluations normalized to [0,1] during training
   Predictions denormalized to [-2000, 2000] centipawns



# Chess ML Evaluation - Testing & Comparison

This section allows you to compare the ML evaluation with traditional evaluation on specific positions.

In [ ]:
# -------------------------------
#    EVALUATION COMPARISON TEST
# -------------------------------

def compare_evaluations(fen_position):
    """Compare ML and traditional evaluations for a given position."""
    board = chess.Board(fen_position)
    
    # ML evaluation
    if ML_MODEL_AVAILABLE:
        ml_eval = evaluate_board_ml(board)
    else:
        ml_eval = "N/A (model not loaded)"
    
    # Traditional evaluation
    trad_eval = evaluate_board_raw(board)
    
    print(f"Position: {fen_position}")
    print(f"Board:\n{board}\n")
    print(f"ML Evaluation:          {ml_eval:+d} cp" if isinstance(ml_eval, int) else f"ML Evaluation:          {ml_eval}")
    print(f"Traditional Evaluation: {trad_eval:+d} cp")
    if isinstance(ml_eval, int):
        print(f"Difference:             {abs(ml_eval - trad_eval):d} cp")
    print("="*60)

# Test on various positions
print("Testing evaluations on different chess positions:\n")

# Starting position
compare_evaluations("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

# After 1.e4
compare_evaluations("rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1")

# Middle game position
compare_evaluations("r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4")

# Endgame position
compare_evaluations("8/5k2/3p4/1p1Pp2p/pP2Pp1P/P4P1K/8/8 b - - 99 50")